Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
STUDENT_ID = ""

---

# MTH793P - Coursework 1

This is a template notebook for the computational exercises of Coursework #1 of the module MTH793P, Advanced Machine Learning. Closely follow the instructions in this template in order to complete the assessment and to obtain full marks. Please only modify cells where you are instructed to do so. Failure to comply may result in unexpected errors that can lead to mark deductions.

As usual, we begin by loading the necessary libraries.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse as sp
from scipy.sparse import linalg

Create two lists *nodes* and *edges* and one NumPy array *weights*. The list *nodes* should contain all names of the nodes in the graph of Coursework 1, Question 1, in alphabetical order. The list edges should include lists that contain the indices of the nodes that are connected by the individual edge. For example, the first edge connects node 'Batman' and node 'Jessica Jones', so the list for this edge should be [0, 3], as 'Batman' is the first entry of the list *nodes* and 'Jessica Jones' is the fourth entry.  The Numpy array *weights* should contain the individual weights assigned to each edge.

In [3]:
nodes = ['Batman', 'Catwoman', 'Wonder woman', 'Jessica Jones', 'Deadpool', 'Spiderman']
edges = [[0,3], [0,1], [0,4], [0,2], [3,4], [2,3], [3,5], [1,2], [1,5], [4,5], [2,5]]
weights = np.array([[4], [81], [16], [64], [64], [36], [49], [49], [1], [49], [1]])
#raise NotImplementedError()

Display your lists and array with the following cell.

In [4]:
print('We consider a graph with nodes/vertices {n}, edges {e} and weights {w}.'.format( \
                            n = nodes, e = edges, w = weights))

We consider a graph with nodes/vertices ['Batman', 'Catwoman', 'Wonder woman', 'Jessica Jones', 'Deadpool', 'Spiderman'], edges [[0, 3], [0, 1], [0, 4], [0, 2], [3, 4], [2, 3], [3, 5], [1, 2], [1, 5], [4, 5], [2, 5]] and weights [[ 4]
 [81]
 [16]
 [64]
 [64]
 [36]
 [49]
 [49]
 [ 1]
 [49]
 [ 1]].


Write a function **construct_incidence_matrix** that takes the lists *nodes*, *edges* and the NumPy array *weights* as arguments. The function should return the incidence matrix *incidence_matrix* that corresponds to the weighted graph. The construction of the incidence matrix is described in the lecture notes.

In [5]:
def construct_incidence_matrix(nodes, edges, weights):
    incidence_matrix = np.zeros((len(edges), len(nodes)))
    for n in range(len(edges)):
        incidence_matrix[n][edges[n][0]] = -np.sqrt(weights[n])
        incidence_matrix[n][edges[n][1]] = np.sqrt(weights[n])
    return incidence_matrix
      
#raise NotImplementedError()

You can test your function with the following cell.

In [6]:
from numpy.testing import assert_array_equal
incidence_matrix = construct_incidence_matrix(nodes, edges, weights)    
print('The incidence matrix of our graph is \n {i}.'.format(i = incidence_matrix.astype(int)))

# The following code is testing the previous code against a specific example    
test_nodes = ['Batman', 'Catwoman', 'Spiderman']
test_edges = [[0, 1], [1, 2]]
test_weights = np.array([81, 1])
test_incidence_matrix = construct_incidence_matrix(test_nodes, test_edges, test_weights)

assert_array_equal(test_incidence_matrix.astype(int), np.array([[-9, 9, 0],[0, -1, 1]]))


The incidence matrix of our graph is 
 [[-2  0  0  2  0  0]
 [-9  9  0  0  0  0]
 [-4  0  0  0  4  0]
 [-8  0  8  0  0  0]
 [ 0  0  0 -8  8  0]
 [ 0  0 -6  6  0  0]
 [ 0  0  0 -7  0  7]
 [ 0 -7  7  0  0  0]
 [ 0 -1  0  0  0  1]
 [ 0  0  0  0 -7  7]
 [ 0  0 -1  0  0  1]].


Next, compute the corresponding graph-Laplacian for the incidence matrix *incidence_matrix* from the previous exercise and store it in a variable named *graph_laplacian*. Follow the definition from the lecture notes.

In [7]:
def laplacian(nodes, edges, weights):
    laplacian_matrix = np.zeros((len(nodes), len(nodes)))
    w = 0
    diagonals = [0]*len(nodes)
    for e in edges:
        laplacian_matrix[e[0]][e[1]] = - weights[w]
        laplacian_matrix[e[1]][e[0]] = - weights[w]
        diagonals[e[0]] += weights[w]
        diagonals[e[1]] += weights[w]
        w += 1
    for n in range(len(nodes)):
        laplacian_matrix[n][n] = diagonals[n]
    return laplacian_matrix
graph_laplacian = laplacian(nodes, edges, weights)
#raise NotImplementedError()

You can test your function with the following cell.

In [8]:
print('The graph Laplacian of our graph is \n {g}.'.format(g = graph_laplacian.astype(int)))

The graph Laplacian of our graph is 
 [[165 -81 -64  -4 -16   0]
 [-81 131 -49   0   0  -1]
 [-64 -49 150 -36   0  -1]
 [ -4   0 -36 153 -64 -49]
 [-16   0   0 -64 129 -49]
 [  0  -1  -1 -49 -49 100]].


We want to use our graph to determine whether a node in the graph belongs to the class "Marvel" or the class "DC". Suppose we are in a semi-supervised setting, where the node "Deadpool" is already labelled $v_{\text{Deadpool}} = 0$ (class "Marvel") and the node "Catwoman" is labelled as $v_{\text{Catwoman}} = 1$ (class "DC"). Here $v$ is the mathematical notation of the label vector. We follow the instructions in the lecture notes and formulate this as a linear system. We can either define appropriate projection matrices or create sub-matrices from the graph-Laplacian by choosing the correct indices. How you set up the linear system is up to you. Store your linear system in a variable named *linear_system* and the right-hand-side in a variable *right_hand_side*.

In [9]:
labels_matrix = np.zeros((2, len(nodes)))
unlabelled_matrix = np.zeros((4, len(nodes)))
y_values = np.array(([1], [0]))

# Known info
labels_matrix[0][1] = 1 #Catwoman
labels_matrix[1][4] = 1 #Deadpool

unlabelled_matrix[0][0] = 1 #Batman
unlabelled_matrix[1][2] = 1 #Wonderwoman
unlabelled_matrix[2][3] = 1 #Jessica Jones
unlabelled_matrix[3][5] = 1 #Spiderman

A_matrix = unlabelled_matrix @ graph_laplacian @ unlabelled_matrix.T
b_vector = unlabelled_matrix @ graph_laplacian @ labels_matrix.T @ y_values

#raise NotImplementedError()

Solve your linear system and store your labels in an array named *remaining_labels*. Create also an array *all_labels* that contains all labels, as well as a boolean array *thresholded_labels* of the same size as *all_labels* with True or False values. An entry should be true if the corresponding entry in *all_labels* is larger than 0.5 and false otherwise.

In [10]:
remaining_labels = -(np.linalg.inv(A_matrix) @ b_vector)

all_labels = np.array((remaining_labels[0], [1], remaining_labels[1], remaining_labels[2], [0], remaining_labels[3]))
thresholded_labels = np.full((6, 1), False)
thresholded_labels[all_labels > 0.5] = True
print(thresholded_labels)
#raise NotImplementedError()

[[ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]]


Check your results with the following cell.

In [11]:
print('The computed labels are {a}. Setting all values above 0.5 to one and'.format( \
        a = all_labels), 'the remaining ones to zero yields {t}.'.format(t = \
        thresholded_labels.astype(int)))

The computed labels are [[0.77273292]
 [1.        ]
 [0.71224896]
 [0.22924953]
 [0.        ]
 [0.12945476]]. Setting all values above 0.5 to one and the remaining ones to zero yields [[1]
 [1]
 [1]
 [0]
 [0]
 [0]].


We conclude this coursework by computing the second eigenvector, i.e. the eigenvector that corresponds to the second smallest eigenvalue, of the graph-Laplacian *graph_laplacian*. Explore how to compute eigenvalues and eigenvectors, in particular of sparse matrices if you have used them.

In [12]:
[eigenvalues, eigenvectors] = linalg.eigs(graph_laplacian)
#raise NotImplementedError()

/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1269: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


Display the second smallest eigenvalue and the corresponding eigenvector. What do you observe?

In [13]:
print('The second smallest eigenvalue of the graph Laplacian is {eval}.'.format(eval = \
        eigenvalues[1].real), 'The corresponding eigenvector is {evec}.T.'.format(evec = \
        eigenvectors[:, 1].T.real))

The second smallest eigenvalue of the graph Laplacian is 33.16375166078193. The corresponding eigenvector is [ 0.39722732  0.48514106  0.32227706 -0.30661694 -0.39542065 -0.50260786].T.


In [14]:
TwoSmallestEvalues = linalg.eigs(graph_laplacian, 2, which = "SM")
print(TwoSmallestEvalues)
#Second smallest
print(TwoSmallestEvalues[0][1])
print(TwoSmallestEvalues[1][:,1])

(array([7.10542736e-15+0.j, 3.31637517e+01+0.j]), array([[ 0.40824829+0.j,  0.39722732+0.j],
       [ 0.40824829+0.j,  0.48514106+0.j],
       [ 0.40824829+0.j,  0.32227706+0.j],
       [ 0.40824829+0.j, -0.30661694+0.j],
       [ 0.40824829+0.j, -0.39542065+0.j],
       [ 0.40824829+0.j, -0.50260786+0.j]]))
(33.16375166078203+0j)
[ 0.39722732+0.j  0.48514106+0.j  0.32227706+0.j -0.30661694+0.j
 -0.39542065+0.j -0.50260786+0.j]
